# finBERT

## Import Key Developments News dataset

In [ ]:
import pandas
import numpy as np

headlines_df = pandas.read_csv('AAPL 2022-2023.csv')
headlines_df = headlines_df[headlines_df['headline'].notna()]
headlines_df = headlines_df[headlines_df['headline'] != '']

headlines_array = np.array(headlines_df)

headlines_list = list(headlines_array[:,4])
stocks_list = list(headlines_array[:, 2])
date_list = list(headlines_array[:, 3])

## Getting the tokenizer and the finBERT model

In [ ]:
!pip install transformers

from transformers import AutoTokenizer, AutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

## Performing inference on the stock market news headlines with the finBERT

In [ ]:
import torch

def chunk_list(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]


STRIDE = 50

model.eval()

sentiment_table = pandas.DataFrame(columns=['headline', 'stock', 'pos', 'neg', 'neutr', 'date'])

n=0
for lines, stocks, dates in zip(chunk_list(headlines_list, STRIDE), chunk_list(stocks_list, STRIDE), 
                                chunk_list(date_list, STRIDE)):
  
  input = tokenizer(lines, padding = True, truncation = True,  return_tensors='pt')
  
  outputs = model(**input)

  prediction = torch.nn.functional.softmax(outputs.logits, dim=-1)

  print(f"{n+1}/{int(len(headlines_list)/STRIDE)}") 

  for headline, stock, pos, neg, neutr, date in zip(lines, stocks, prediction[:, 0].tolist(), 
                                                    prediction[:, 1].tolist(), prediction[:, 2].tolist(), dates): 
    sentiment_table.loc[len(sentiment_table)] = {'headline': headline, 'stock': stock, 'pos': pos, 'neg': neg, 
                                                 'neutr': neutr, 'date':date}
   
  n+=1

## Save the sentiment dataframe as a CSV file

In [ ]:
sentiment_table = pandas.DataFrame(sentiment_table)
print(sentiment_table)

sentiment_table.to_csv('AAPL_sentiment_table.csv', index=False)

In [ ]:
# Group by date and calculate average for pos, neg, neutr columns and take first value for stock column
sent_for_training = sentiment_table.groupby('date').agg({'stock': 'first', 'pos': 'mean', 'neg': 'mean', 
                                                         'neutr': 'mean'})

# Sort by date (index) in ascending order
sent_for_training.sort_index(ascending=True, inplace=True)
# Save the new DataFrame to a csv file
sent_for_training.to_csv('AAPL_2022_2023_FOR_TRAINING.csv', index=False)

print(sent_for_training)

# Processing the 2 datasets

## utils

In [ ]:
!pip install pandas-datareader

In [ ]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from pandas_datareader import data as data_reader
import yfinance as yf

yf.pdr_override()

from tqdm import tqdm_notebook, tqdm
from collections import deque
from sklearn.preprocessing import MinMaxScaler

## Loading the two datasets

In [ ]:
stock_name = 'AAPL'
start_date = '2023-1-1'
end_date = '2023-2-28'

date_range = pd.date_range(start=start_date, end=end_date)

dataset = data_reader.get_data_yahoo(stock_name, start=start_date, end=end_date)
sentiment_table = pd.read_csv(stock_name + '_sentiment_table.csv')

## Create TRAINING dataset

In [ ]:
# Add 'virality' feature which counts the number of news itmes there are per day
sentiment_table['virality'] = sentiment_table.groupby('date')['date'].transform('size')

# Average sentiment per day
sent_for_training = sentiment_table.groupby('date').agg({'stock': 'first', 'pos': 'mean', 'neg': 'mean', 
                                                         'neutr': 'mean', 'virality': 'first'})


sent_for_training.sort_index(ascending=True, inplace=True)
merged_df = dataset.copy()
merged_df['pos'] = np.nan
merged_df['neg'] = np.nan
merged_df['neutr'] = np.nan
merged_df['virality'] = np.nan

# add sentiment from days when market is open and create the closed days dataset of sentiment
closed_df = pd.DataFrame()
closed_rows =[]
for date in sent_for_training.index:
  # Check if the date exists in merged_df
    if date in merged_df.index:
      # Copy 'pos', 'neg', 'neutr' values from sent_for_training to merged_df
        merged_df.loc[date, 'pos'] = sent_for_training.loc[date, 'pos']
        merged_df.loc[date, 'neg'] = sent_for_training.loc[date, 'neg']
        merged_df.loc[date, 'neutr'] = sent_for_training.loc[date, 'neutr']
        merged_df.loc[date, 'virality'] = sent_for_training.loc[date, 'virality']
    else:
        # If the date is not in merged_df, add it to 'closed' DataFrame
        closed_row = sent_for_training.loc[[date]]
        closed_rows.append(closed_row)
if len(closed_rows) > 0:
  closed_df = pd.concat(closed_rows)

# add sentiment from news when market was closed
counting = False
closed_days = []

for date in date_range:
    date_str = date.strftime("%Y-%m-%d")

    if counting and date_str in merged_df.index: 
        if pd.isna(merged_df.loc[date_str, 'pos']):
            merged_df.loc[date_str, ['pos', 'neg', 'neutr', 'virality']] = 
            closed_df.loc[closed_days].mean(numeric_only=True)
        else:
            merged_df.loc[date_str, ['pos', 'neg', 'neutr', 'virality']] += 
            closed_df.loc[closed_days].mean(numeric_only=True)
        closed_df = closed_df.drop(closed_days)
        closed_days = []
        counting = False
        
    if date_str in closed_df.index:
        counting = True
        closed_days.append(date_str)

merged_df.reset_index(drop=True, inplace=True)

# set to baseline 0 sentiment if first day has no news
merged_df.iloc[0] = merged_df.iloc[0].fillna(0)
# set the sentiment to previous values on days when no news was released
merged_df['pos'].fillna(method='ffill', inplace=True)
merged_df['neg'].fillna(method='ffill', inplace=True)
merged_df['neutr'].fillna(method='ffill', inplace=True)
merged_df['virality'].fillna(method='ffill', inplace=True)

#normalise the data
scaler = MinMaxScaler()
# Scale "Open", "High", "Low", "Close", "Adj Close" columns together
merged_df[['Open', 'High', 'Low', 'Close', 'Adj Close']] = scaler.fit_transform(merged_df[['Open', 'High', 
                                                                                'Low', 'Close', 'Adj Close']])
# Scale "Volume", "pos", "neg", "neutr" columns individually
for col in ['Volume', 'virality']:
    merged_df[[col]] = scaler.fit_transform(merged_df[[col]])

print(merged_df.head(5))

## Save training dataset

In [ ]:
merged_df.to_csv(stock_name + '_2022_2023_Training.csv', index=True)

# S-DRQN model

## Installing

In [ ]:
!pip install pandas-datareader

## utils

In [ ]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from pandas_datareader import data as data_reader

from tqdm import tqdm_notebook, tqdm
from collections import deque

## S-DRQN algorithm

### AI Day Trader class definition

In [ ]:
class AIDayTrader:

    def __init__(self, state_shape, action_space=3):  # Stay, Buy, Sell

        self.state_shape = state_shape
        self.action_space = action_space
        self.memory = deque(maxlen=2000)
        self.inventory = []
        self.model_name = "AI_Day_Trader"

        self.gamma = 0.99
        self.epsilon = 1.0
        self.epsilon_final = 0.01
        self.epsilon_decay = 0.995

        # Exponential Decay of Learning Rate
        lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.0005, 
                                                                     decay_steps=250, decay_rate=0.99)

        self.lr = 0.00025
        self.model = self.model_builder()

        self.profit = 0
        self.spent = 0
        self.cumulative_reward = 0
        self.last_action_was_buy = False

    def model_builder(self):

        model = tf.keras.models.Sequential()
        model.add(tf.keras.layers.Dense(256, activation='relu', input_shape=[self.state_shape]))
        model.add(tf.keras.layers.Dense(256, activation='relu'))
        model.add(tf.keras.layers.Dense(256, activation='relu'))
        model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))
        model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=self.lr), 
                      metrics=['accuracy', 'mae'])

        return model

    def trade(self, state):
        state = np.reshape(state, [1, self.state_shape])

        if random.random() <= self.epsilon:
            if self.last_action_was_buy:
                return random.choice([0, 2])  # Force a hold or sell if the last action was a buy
            if len(self.inventory) == 0:
                return random.choice([0, 1])  # Force a hold or buy if inventory is empty
            else:
                return random.randrange(self.action_space)

        actions = self.model.predict(state)[0]
        action = np.argmax(actions)

        if self.last_action_was_buy and action == 1:
            sorted_indices = np.argsort(actions)[::-1]
            action = sorted_indices[1]  # Force a hold or sell if the last action was a buy
        if len(self.inventory) == 0 and action == 2:
            sorted_indices = np.argsort(actions)[::-1]
            action = sorted_indices[1]  # Force a hold or buy if inventory is empty

        return action

    def batch_train(self, batch_size):
        batch = []

        for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
            batch.append(self.memory[i])

        for state, action, reward, next_state, done in batch:
            re_next = np.reshape(next_state, [1, self.state_shape])
            re_state = np.reshape(state, [1, self.state_shape])

            t_reward = reward

            if not done:
                t_reward = reward + self.gamma * np.amax(self.model.predict(re_next)[0])

            target = self.model.predict(re_state)
            target[0, action] = t_reward

            self.model.fit(re_state, target, epochs=1, verbose=1)

        if self.epsilon > self.epsilon_final:
            self.epsilon *= self.epsilon_decay

### Helper functions

In [ ]:
def stocks_price_format(n):
    if n < 0:
        return "- $ {0:2f}".format(abs(n))
    else:
        return "$ {0:2f}".format(abs(n))


def dataset_loader(stock_name, start='2022-1-1', end='2023-1-1'):
    dataset = data_reader.get_data_yahoo(stock_name, start=start, end=end)

    return dataset


def state_creator(data, timestep):
    state = data.iloc[timestep]
    return np.array(state)  # Return as a 2D array

### Hyperparameters...

In [ ]:
# ticker
stock_name = "AAPL"

data = pd.read_csv('AAPL_2022_2023_Training.csv')
data = data.drop(data.columns[0], axis=1)

episodes = 2500
batch_size = 16

data_samples = len(data)
state_shape = (len(data.columns))

### Model

In [ ]:
trader = AIDayTrader(state_shape)

In [ ]:
trader.model.summary()

### Train

In [ ]:
%%capture cap


all_profit = []
all_spent = []


for episode in range(1, episodes + 1):
    print("Episode: {}/{}".format(episode, episodes))

    state = state_creator(data, 0)

    trader.inventory = []

    # Keep track of whether a buy or a sell action was made
    buy_action = False
    sell_action = False
    trader.last_action_was_buy = False

    reward_t1 = 0

    for t in tqdm(range(data_samples-1)):
        print("Day: {}/{}".format(t+1, data_samples-1))

        action = trader.trade(state)

        next_state = state_creator(data, t+1)

        reward_t = 0  # Staying

        if action == 1:  # Buying
            if not trader.last_action_was_buy:  # Only allow buying if the last action was not a buy
                trader.inventory.append(data.iloc[t])
                trader.spent += data['Open'][t]
                reward_t = data['Close'][t] - data['Open'][t]
                print("AI Trader bought: ", stocks_price_format(data['Open'][t]))
                trader.last_action_was_buy = True
                buy_action = True

        if action == 2 and len(trader.inventory) > 0:  # Selling
            buy_price = trader.inventory.pop(0)
            profit = data['Open'][t] - buy_price['Open']
            reward_t = -(data['Close'][t] - data['Open'][t])
            trader.profit += reward_t + profit
            print("AI Trader sold: ", stocks_price_format(data['Open'][t]), " Profit: " + 
                  stocks_price_format(reward_t))
            trader.last_action_was_buy = False
            sell_action = True

        if action == 0:  # Staying
            if len(trader.inventory) > 0:
                reward_t = data['Close'][t] - data['Open'][t]
            else:
                reward_t = -(data['Close'][t] - data['Open'][t])

        if t == data_samples - 2:
            done = True
            if len(trader.inventory) > 0:
                buy_price = trader.inventory.pop(0)
                reward_t += -(data['Close'][t] - data['Open'][t])
                trader.profit += reward_t
                print("AI Trader sold: ", stocks_price_format(data['Close'][t]), " Profit: " + 
                      stocks_price_format(reward_t))
            if (buy_action == False) or (sell_action == False):
                reward_t = -1
        else:
            done = False

        reward_t += reward_t1

        if (reward_t*reward_t1 > 0) and (reward_t1 != 0):
            trader.cumulative_reward = np.log(reward_t / reward_t1)
        else:
            trader.cumulative_reward = -1

        reward_t1 = reward_t

        trader.memory.append((state, action, trader.cumulative_reward, next_state, done))
        state = next_state

        if done:

            print("########################")
            print("TOTAL PROFIT: {}".format(trader.profit))
            print("TOTAL SPENT: {}".format(trader.spent))
            print("########################")

            all_profit.append(trader.profit)
            all_spent.append(trader.spent)

            trader.profit = 0
            trader.spent = 0

            df = pd.DataFrame(all_profit, columns=["Total_Profit"])
            df.to_csv("total_profit.csv", index=False)
            fd = pd.DataFrame(all_spent, columns=["Total_Spending"])
            fd.to_csv("total_spent.csv", index=False)

            if len(trader.memory) > batch_size:
                trader.batch_train(batch_size)

    if episode % 100 == 0:
        trader.model.save("ai_day_trader_{}.h5".format(episode))

### Save output shell

In [ ]:
f = open("financial DRQN verbose.txt", "w") 
print(cap, file=f)
f.close()

## Plot

### Plot profit

In [ ]:
data = pd.read_csv('total_profit.csv')
data = np.array(data)

total = list(data[:,0])

# Calculate moving average
window_size = 100
moving_average = np.convolve(total, np.ones(window_size) / window_size, mode='valid')

# Plotting
plt.plot(range(len(total)), total, marker='o', label='Actual Values')
plt.plot(range(window_size - 1, len(total)), moving_average, label='Moving Average')
plt.xlabel('Episode')
plt.ylabel('Profit')
plt.title('Profit per episode')
plt.legend()
plt.ylim(-2, 1)
plt.grid(True)
plt.show()

### Plot % return on investment

In [ ]:
profit_data = pd.read_csv('total_profit.csv')
spent_data = pd.read_csv('total_spent.csv')

# Convert to numpy arrays
profit = np.array(profit_data['Total_Profit'])
spent = np.array(spent_data['Total_Spending'])

# Calculate percentage profit
percentage_profit = np.empty(len(profit))
for i in range(len(profit)):
    if (profit[i] != 0) and (spent[i] != 0):
        percentage_profit[i] = np.round((profit[i] / spent[i]) * 100, 3)
    else:
        percentage_profit[i] = 0

# Calculate moving average
window_size = 100
moving_average = np.convolve(percentage_profit, np.ones(window_size) / window_size, mode='valid')

# Plotting
plt.plot(range(len(percentage_profit)), percentage_profit, marker='o', label='Actual Values')
plt.plot(range(window_size - 1, len(percentage_profit)), moving_average, label='Moving Average')
plt.xlabel('Episode')
plt.ylabel('Profit (%)')
plt.title('ROI')
plt.legend()
plt.ylim(-30, 30)
plt.grid(True)
plt.show()

# Evaluate

## util

In [ ]:
!pip install pandas-datareader

import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from pandas_datareader import data as data_reader

from tqdm import tqdm_notebook, tqdm
from collections import deque


## Load model

In [ ]:
from tensorflow.keras.models import load_model

path = 'ai_day_trader_1500.h5'
model = load_model(path)

## Load test dataset

In [ ]:
stock_name = 'AAPL'
test_df = pd.read_csv(stock_name + '_test_data.csv')
test_df = test_df.drop(test_df.columns[0], axis=1)

## Predict

In [ ]:
test_data = np.array(test_df)

predictions = model.predict(test_data)

In [ ]:
np.savetxt('Predictions.csv', predictions, delimiter=',')

## Calculate Profit

In [ ]:
spent = 0
profit = 0
reward = 0
inventory = []
last_action_was_buy = False

for i in range (len(predictions)-1):
  action = np.argmax(predictions[i])
  print(action)
  action_taken = False
  bought = False

  while not action_taken:

    if action == 1:  # Buying
      if not last_action_was_buy:  # Only allow buying if the last action was not a buy
        inventory.append(test_df.iloc[i])
        spent += test_df['Open'][i]
        print("Day: {}/{}".format(i+1, len(predictions)), "AI Trader bought: ", 
              stocks_price_format(test_df['Open'][i]))
        last_action_was_buy = True
        action_taken = True
      else:
        sorted_indices = np.argsort(predictions[i])[::-1]
        action = sorted_indices[1]
      
    if action == 2:
      if len(inventory) > 0:  # Selling
        buy_price = inventory.pop(0)
        profit += test_df['Open'][i] - buy_price['Open']
        print("Day: {}/{}".format(i+1, len(predictions)), "AI Trader sold: ", 
              stocks_price_format(test_df['Open'][i]), " Profit: " + stocks_price_format(profit))
        last_action_was_buy = False
        action_taken = True
      else:
        sorted_indices = np.argsort(predictions[i])[::-1]
        action = sorted_indices[1]

    if action == 0:
      action_taken = True


print('Spent', spent)
print('Profit', profit)
print('ROI', np.round((profit / spent) * 100, 3))